<a href="https://colab.research.google.com/github/devyulbae/Kaggle/blob/main/Kag_BinClass_for_Bank_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# readme
# for Kaggle competition
# name: Binary Classification with a Bank Churn Dataset
# period: 2024.Jan
#
#
#

In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Explore Dataset

In [ ]:
# google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Dealing With Categorical Data

In [ ]:
# Data Visualization

In [ ]:
# Building Our Models

In [ ]:
# Submission